<a href="https://colab.research.google.com/github/rohangrge/Machine-Learning-with-Spark-Streaming/blob/branch1/BD_corr_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from scipy.stats import chisquare
import nltk
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize 
from nltk.corpus import stopwords
nltk.download('punkt')
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer(lowercase=False)
stop_words =set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
df = pd.read_csv("train.csv")

In [3]:
df.head

<bound method NDFrame.head of                                                Subject  ... Spam/Ham
0                                   transfers from ees  ...     spam
1                            fw : re ivanhoe e . s . d  ...     spam
2                re : enerfin meter 980439 for 10 / 00  ...      ham
3                                    meoh plant status  ...      ham
4                                      re : tenaska iv  ...     spam
...                                                ...  ...      ...
30339                        fw : re ivanhoe e . s . d  ...     spam
30340                                        wellheads  ...      ham
30341                                    re : releases  ...     spam
30342  start date : 2 / 6 / 02 ; hourahead hour : 24 ;  ...     spam
30343                                    re : releases  ...     spam

[30344 rows x 3 columns]>

In [4]:
df.columns

Index(['Subject', 'Message', 'Spam/Ham'], dtype='object')

In [5]:
spec_chars = ["!",'"',"#","%","&","'","(",")",
              "*","+",",","-",".","/",":",";","<",
              "=",">","?","@","[","\\","]","^","_",
              "`","{","|","}","~","–"]
for char in spec_chars:
    df['Message'] = df['Message'].str.replace(char, ' ')
    df['Subject'] = df['Subject'].str.replace(char, ' ')
df['Message'] = df['Message'].str.split().str.join(" ")
df['Subject'] = df['Subject'].str.split().str.join(" ")

In [6]:
df['Subject_tokenized'] = df['Subject'].apply(word_tokenize)
df['Message_tokenized'] = df['Message'].apply(lambda sentence: nltk.word_tokenize(str(sentence)))

In [7]:
df.head()

,Subject,Message,Spam/Ham,Subject_tokenized,Message_tokenized
0,transfers from ees,attached is the latest version of the cost cen...,spam,"[transfers, from, ees]","[attached, is, the, latest, version, of, the, ..."
1,fw re ivanhoe e s d,fyi kim original message from frazier perry se...,spam,"[fw, re, ivanhoe, e, s, d]","[fyi, kim, original, message, from, frazier, p..."
2,re enerfin meter 980439 for 10 00,it did but tetco prorated the flow between the...,ham,"[re, enerfin, meter, 980439, for, 10, 00]","[it, did, but, tetco, prorated, the, flow, bet..."
3,meoh plant status,the methanol plant has determined extensive re...,ham,"[meoh, plant, status]","[the, methanol, plant, has, determined, extens..."
4,re tenaska iv,i tried calling you this am but your phone rol...,spam,"[re, tenaska, iv]","[i, tried, calling, you, this, am, but, your, ..."


In [8]:
df["Subject_tfidf"] = v.fit_transform(df['Subject_tokenized'].astype('U').values)
df["Message_tfidf"] = v.fit_transform(df['Message_tokenized'].astype('U').values)

In [9]:
df.head()

,Subject,Message,Spam/Ham,Subject_tokenized,Message_tokenized,Subject_tfidf,Message_tfidf
0,transfers from ees,attached is the latest version of the cost cen...,spam,"[transfers, from, ees]","[attached, is, the, latest, version, of, the, ...","(0, 2337)\t0.5799269978885242\n (0, 2855)\t...","(0, 55967)\t0.08907587912053541\n (0, 52674..."
1,fw re ivanhoe e s d,fyi kim original message from frazier perry se...,spam,"[fw, re, ivanhoe, e, s, d]","[fyi, kim, original, message, from, frazier, p...","(0, 2337)\t0.5799269978885242\n (0, 2855)\t...","(0, 55967)\t0.08907587912053541\n (0, 52674..."
2,re enerfin meter 980439 for 10 00,it did but tetco prorated the flow between the...,ham,"[re, enerfin, meter, 980439, for, 10, 00]","[it, did, but, tetco, prorated, the, flow, bet...","(0, 2337)\t0.5799269978885242\n (0, 2855)\t...","(0, 55967)\t0.08907587912053541\n (0, 52674..."
3,meoh plant status,the methanol plant has determined extensive re...,ham,"[meoh, plant, status]","[the, methanol, plant, has, determined, extens...","(0, 2337)\t0.5799269978885242\n (0, 2855)\t...","(0, 55967)\t0.08907587912053541\n (0, 52674..."
4,re tenaska iv,i tried calling you this am but your phone rol...,spam,"[re, tenaska, iv]","[i, tried, calling, you, this, am, but, your, ...","(0, 2337)\t0.5799269978885242\n (0, 2855)\t...","(0, 55967)\t0.08907587912053541\n (0, 52674..."


In [10]:
df.dtypes

Subject              object
Message              object
Spam/Ham             object
Subject_tokenized    object
Message_tokenized    object
Subject_tfidf        object
Message_tfidf        object
dtype: object

In [ ]:
#df1 = df.apply(lambda x: tuple(x) if type(x)!= str else tuple([x]))
df["Subject_tokenized"].apply(lambda x : pd.factorize(x))
df["Message_tokenized"].apply(lambda x : pd.factorize(x))
df["Message_tokenized"].corr(df["Subject_tokenized"], method="pearson")

In [ ]:
df=df.apply(lambda x : pd.factorize(x)[0])+1

pd.DataFrame([chisquare(df[x].values,f_exp=df.values.T,axis=1)[0] for x in df])